# Preprocess of the dataset ROBOFLOW

#### Create the binary masks for each image from the annotation file (COCO standard)

In [ ]:
import json
import os
import cv2
import numpy as np

def create_mask_from_annotations(json_path, output_mask_dir): 
    os.makedirs(output_mask_dir, exist_ok=True)

    # === UPLOAD FILE JSON COCO ===
    with open(json_path, "r") as f:
        coco = json.load(f)

    image_info = {img['id']: (img['file_name'], img['width'], img['height']) for img in coco['images']}

    # temporary dictionary to hold masks
    masks = {}  # image_id -> mask (np.ndarray)

    # For each image, create a mask
    for ann in coco['annotations']:
        image_id = ann['image_id']
        bbox = ann['bbox']  # [x, y, width, height]
        x, y, w, h = map(int, bbox)

        file_name, width, height = image_info[image_id]

        # If the image_id is not in masks, create a new mask all black (no annotations)
        if image_id not in masks:
            mask = np.zeros((height, width), dtype=np.uint8)
            masks[image_id] = mask
        else:
            mask = masks[image_id]

        # Draw a white rectangle within the image bounds
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, thickness=-1)  # filled rectangle

    # Save masks to the output directory
    for image_id, mask in masks.items():
        file_name, _, _ = image_info[image_id]
        base_name = os.path.splitext(file_name)[0]
        out_path = os.path.join(output_mask_dir, f"{base_name}.png")
        cv2.imwrite(out_path, mask)

    # if image not in masks create a mask all black
    for image_id, (file_name, width, height) in image_info.items():
        if image_id not in masks:
            mask = np.zeros((height, width), dtype=np.uint8)
            base_name = os.path.splitext(file_name)[0]
            out_path = os.path.join(output_mask_dir, f"{base_name}.png")
            cv2.imwrite(out_path, mask)


In [ ]:
input_folder = "robodata_original"
output_folder = "robodata_original"

for folder in ["test", "train", "valid"]:
    json_path = os.path.join(input_folder, folder,"_annotations.coco.json")
    output_mask_dir = os.path.join(output_folder, folder, "masks")
    create_mask_from_annotations(json_path, output_mask_dir)


#### Central crop of 1024x1024 of the orginal image 
Before cropping remove 200px from each side because there are other objects in some images

In [ ]:
import cv2
import numpy as np

def center_crop_gear(image, mask, output_size=1024):
    # Remove 200 pixels from each side
    image = image[200:-200, 200:-200]
    mask = mask[200:-200, 200:-200]

    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        raise ValueError("Nessun contorno trovato.")

    # Find the largest contour, i.e. the gear
    main_contour = max(contours, key=cv2.contourArea)

    # Computethe centroid of the main contour
    M = cv2.moments(main_contour)
    if M["m00"] == 0:
        raise ValueError("Momento nullo.")
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])

    # Compute the cropping box
    half = output_size // 2
    top = max(cy - half, 0)
    bottom = min(cy + half, image.shape[0])
    left = max(cx - half, 0)
    right = min(cx + half, image.shape[1])

    # crop the image and mask
    cropped = image[top:bottom, left:right]
    cropped_mask = mask[top:bottom, left:right]

    # Add padding to ensure the output size is output_size x output_size
    final = np.ones((output_size, output_size, 3), dtype=np.uint8) * 255  # white background
    y_offset = (output_size - cropped.shape[0]) // 2
    x_offset = (output_size - cropped.shape[1]) // 2
    final[y_offset:y_offset + cropped.shape[0], x_offset:x_offset + cropped.shape[1]] = cropped

    # Add padding to mask
    final_mask = np.ones((output_size, output_size), dtype=np.uint8) * 255  # white background
    final_mask[y_offset:y_offset + cropped_mask.shape[0], x_offset:x_offset + cropped_mask.shape[1]] = cropped_mask
    
    return final, final_mask


In [ ]:
import os

input_folder = "robodata_original"
output_folder = "robodata_cropped"
os.makedirs(output_folder, exist_ok=True)

for folder in ["test", "train", "valid"]:
    input_subfolder = os.path.join(input_folder, folder)
    output_subfolder = os.path.join(output_folder, folder)
    os.makedirs(output_subfolder, exist_ok=True)
    os.makedirs(os.path.join(output_subfolder, "masks"), exist_ok=True)

    for filename in os.listdir(input_subfolder):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(input_subfolder, filename))
            mask = cv2.imread(os.path.join(input_subfolder, "masks", filename.replace(".jpg", ".png")), cv2.IMREAD_GRAYSCALE)
            
            cropped_img, cropped_mask = center_crop_gear(img, mask, 1024)
            cv2.imwrite(os.path.join(output_subfolder, filename), cropped_img)
            cv2.imwrite(os.path.join(output_subfolder, "masks", filename), cropped_mask)


#### Create the median image for each gear
To do so we have already centered the gear in the image, we will take the image and rotate it of 1/16 of 360° because there are 16 teeth in the gear. After the rotation we use the cv2.findTransformECC fucntion that find the best euclidian transformation (transation and rotation) to best match the original image and the rotated one. In this way the two images will now overlap. <br>
In this way we will create the median image, i.e. the image that represents the "perfect" teeth of the gear as it is the median over all the teaths of the gear.

In [ ]:
import cv2
import numpy as np
BG_COLOR = (255, 255, 255)

def align_img(img, target_img):
    """Allinea l'immagine di input all'immagine di destinazione usando ECC."""
    ref_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    tgt_gray = cv2.cvtColor(target_img, cv2.COLOR_BGR2GRAY)

    warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Optimization criteria for ECC (Enhanced Correlation Coefficient)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 2000, 1e-5)

    # Align the image with ECC
    _, warp_matrix = cv2.findTransformECC(ref_gray, tgt_gray, warp_matrix, cv2.MOTION_EUCLIDEAN, criteria)

    # Apply the transformation to the mask
    aligned_img = cv2.warpAffine(img, warp_matrix, (target_img.shape[1], target_img.shape[0]), flags=cv2.INTER_NEAREST, borderValue=BG_COLOR)

    return aligned_img

def create_rotated_median(image_path, num_rotations=16):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rotated_images = []

    # Precompute rotation matrices to save time
    rotation_matrices = [
        cv2.getRotationMatrix2D(center, i * (360 / num_rotations), 1.0)
        for i in range(num_rotations)
    ]
    flag=True
    for rot_mat in rotation_matrices:
        rotated = cv2.warpAffine(image, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=BG_COLOR)
        try:
            aligned = align_img(rotated, image)
        except:
            print(f"Alignment failed for {image_path}")
            aligned = image
            flag=False
        rotated_images.append(aligned)

    # Stack and compute median along axis 0 (per pixel)
    if flag:
        stacked = np.stack(rotated_images).astype(np.float32)
        median_image = np.median(stacked, axis=0).astype(np.uint8)
        return median_image


In [ ]:
import os 

# folder = "robodata_cropped"

for folder in ["test", "train", "valid"]:
    folder = f"robodata_cropped/{folder}"
    os.makedirs(os.path.join(folder, "median"), exist_ok=True)

    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            
            if os.path.exists(os.path.join(folder, "median", filename)):
                print(f"Median image already exists for {filename}")
                continue

            img_path = os.path.join(folder, filename)
            median_result = create_rotated_median(img_path)

            if median_result is not None:
                cv2.imwrite(os.path.join(folder, "median", filename), median_result)
            else:
                print(f"Median image creation failed for {img_path}")

Median image already exists for teeth_broken_25726_jpg.rf.1dbc9520dfa94b0f93f84e8eeb32a260.jpg
Median image already exists for teeth_broken_25729_jpg.rf.7add0d9431740d8fe3ff6882c2689d35.jpg
Median image already exists for teeth_broken_25735_jpg.rf.5cf4f0b01ee828626d156c8830f082e5.jpg
Median image already exists for teeth_broken_25740_jpg.rf.d6f00132c9be5301cb6ed4133ab5937a.jpg
Median image already exists for teeth_broken_25756_jpg.rf.93c3bfc0a90f8d281f9100243624711b.jpg
Median image already exists for teeth_broken_25759_jpg.rf.311cef41a2f7d262c7bf120e5fee7b0d.jpg
Median image already exists for teeth_broken_25774_jpg.rf.ed459039788ca837552cd59351841b6a.jpg
Median image already exists for teeth_broken_25779_jpg.rf.ce57d0849da8a8a1c36475d40542568f.jpg
Median image already exists for teeth_broken_25822_jpg.rf.71b5dad464ba32533aeceb358b51acec.jpg
Median image already exists for teeth_broken_25825_jpg.rf.c6814849da7ab5e9233ddd92bdcbb7c8.jpg
Median image already exists for teeth_broken_25831

#### Create standard notation A B label

In [ ]:
# save: median in A, imgs in B, masks in label
input_folder = "robodata_cropped"
output_folder = "robodata_standardized"

os.makedirs(output_folder, exist_ok=True)
for out_sub_folder in ["A", "B", "label"]:
    os.makedirs(os.path.join(output_folder, out_sub_folder), exist_ok=True)

for subfolder in ["test", "train", "valid"]:
    subfolder_med_path = os.path.join(input_folder, subfolder, "median")

    for filename in os.listdir(subfolder_med_path):
        if filename.endswith(".jpg"):
            # Copy the median image in folder A
            src_path = os.path.join(subfolder_med_path, filename)
            dst_path = os.path.join(output_folder, "A", filename.replace(".jpg", ".png"))
            cv2.imwrite(dst_path, cv2.imread(src_path))

            # Copy the image in folder B
            src_path = os.path.join(input_folder, subfolder, filename)
            dst_path = os.path.join(output_folder, "B", filename.replace(".jpg", ".png"))
            cv2.imwrite(dst_path, cv2.imread(src_path))

            # Copia la maschera nella cartella label
            mask_src_path = os.path.join(input_folder, subfolder, "masks", filename)
            mask_dst_path = os.path.join(output_folder, "label", filename.replace(".jpg", ".png"))
            cv2.imwrite(mask_dst_path, cv2.imread(mask_src_path))
    

# Augment and reduce dataset

Split the dataset in train/val/test, then divide each image of 1024x1024 in 4 images 512x512, then we'll apply data augmentation

In [ ]:
import os
import random

def split_filenames(input_dir, output_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
    os.makedirs(output_dir, exist_ok=True)

    filenames = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

    # Shuffle for randomness
    random.seed(seed)
    random.shuffle(filenames)

    total = len(filenames)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    # Split the list
    train_files = filenames[:train_end]
    val_files = filenames[train_end:val_end]
    test_files = filenames[val_end:]

    # Rename files
    train_files = ['train_'+line for line in train_files]
    val_files = ['val_'+line for line in val_files]
    test_files = ['test_'+line for line in test_files]

    # Write to .txt files that contain the filenames for each split
    with open(os.path.join(output_dir, 'train.txt'), 'w') as f:
        f.write('\n'.join(train_files))

    with open(os.path.join(output_dir, 'val.txt'), 'w') as f:
        f.write('\n'.join(val_files))

    with open(os.path.join(output_dir, 'test.txt'), 'w') as f:
        f.write('\n'.join(test_files))

    print(f"Saved train/val/test splits to: {output_dir}")

def rename_files(folder_dir):
    for split in ['train', 'val', 'test']:
        with open(os.path.join(folder_dir, 'list', split+'.txt'), 'r') as f:
            lines = f.read().splitlines()
        for file_name in lines:
            old_file_name = file_name.split('_', 1)[1]
            for sub_folder in ['A', 'B', 'label']:
                old_path = os.path.join(folder_dir, sub_folder, old_file_name)
                new_path = os.path.join(folder_dir, sub_folder, split+'_'+old_file_name)

                os.rename(old_path, new_path)



In [ ]:
import shutil

input_folder = "robodata_standardized"
output_folder = "Dataset_ROBOFLOW"

# Copy the folder and all its contents
shutil.copytree(input_folder, output_folder)

'Dataset_ROBOFLOW'

In [7]:
tmp_input = os.path.join(output_folder, "A").replace('\\', '/')
tmp_output = os.path.join(output_folder, "list").replace('\\', '/')

split_filenames(input_dir=tmp_input, output_dir=tmp_output)
rename_files(folder_dir=output_folder)

Saved train/val/test splits to: Dataset_ROBOFLOW/list


Define the function to divide in 4 the images and to do data augmentation

In [ ]:
import os
import random
import cv2


INPUT_SIZE = 1024
OUT_SIZE = INPUT_SIZE//2

def crop_and_save_image(image_path, output_dir):
    img_name = os.path.basename(image_path).replace('\\', '/')
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image from: {image_path}")

    height, width, _ = image.shape

    # Check for expected dimensions
    if height != INPUT_SIZE or width != INPUT_SIZE:
        raise ValueError(f"Expected image size {INPUT_SIZE}x{INPUT_SIZE}, got {width}x{height}")

    # Define coordinates for the 4 crops
    crops = [
        (0, 0),                 # Top-left
        (0, OUT_SIZE),          # Top-right
        (OUT_SIZE, 0),          # Bottom-left
        (OUT_SIZE, OUT_SIZE)    # Bottom-right
    ]

    for i, (y, x) in enumerate(crops):
        cropped = image[y:y+OUT_SIZE, x:x+OUT_SIZE]
        output_path = os.path.join(output_dir, f"{img_name[:-4]}_{i+1}.png").replace('\\', '/')
        cv2.imwrite(output_path, cropped)
        print(f"Saved: {output_path}")

def update_filenames_list(input_folder, output_folder):
    '''
    In input folder A with the median images, in output the list folder where the 3 .txt files will be
    '''
    for split in ['train', 'val', 'test']:
        output_file_path = os.path.join(output_folder, split+'.txt')
        open(output_file_path, 'w').close()

    for img_name in os.listdir(input_folder):
        for split in ['train', 'val', 'test']:
            if split in img_name[:5]:
                output_file_path = os.path.join(output_folder, split+'.txt')
                with open(output_file_path, 'a') as g:  # 'a' means append
                    g.write(img_name + '\n')



create the folder "_splitted" where we'll save the divided images, then define the .txt files in the "list" folder

In [9]:
input_dir = "Dataset_ROBOFLOW"
output_dir = "Dataset_ROBOFLOW_cropped"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)
for folder in ["A", "B", "label", "list"]:
    fold_path = os.path.join(output_dir, folder).replace('\\', '/')
    os.makedirs(fold_path, exist_ok=True)

for filename in os.listdir(os.path.join(input_dir, 'A')):
    for folder in ["A", "B", "label"]:
        file_path = os.path.join(input_dir, folder, filename).replace('\\', '/')
        output_sub_dir = os.path.join(output_dir, folder).replace('\\', '/')
        crop_and_save_image(image_path=file_path, output_dir=output_sub_dir)

update_filenames_list(input_folder=os.path.join(output_dir, 'A'), output_folder=os.path.join(output_dir, 'list'))

Saved: Dataset_ROBOFLOW_cropped/A/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_1.png
Saved: Dataset_ROBOFLOW_cropped/A/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_2.png
Saved: Dataset_ROBOFLOW_cropped/A/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_3.png
Saved: Dataset_ROBOFLOW_cropped/A/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_4.png
Saved: Dataset_ROBOFLOW_cropped/B/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_1.png
Saved: Dataset_ROBOFLOW_cropped/B/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_2.png
Saved: Dataset_ROBOFLOW_cropped/B/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_3.png
Saved: Dataset_ROBOFLOW_cropped/B/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_4.png
Saved: Dataset_ROBOFLOW_cropped/label/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_1.png
Saved: Dataset_ROBOFLOW_cropped/label/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6806fb7_2.png
Saved: Dataset_ROBOFLOW_cropped/label/test_34735_jpg.rf.64a1fb8d516e877f479d1074a6

Create a new folder "_augmented" where we store the images after applying the following augmentation methods in 3 phases:
1) All possible combinations:
   - Rotations (90,180,270)
   - save the original image (0 degree)


In [ ]:
import os
import cv2
import numpy as np

BG_img = 255
BG_mask = 0

def save_and_apply_transformations(img_A_path, img_B_path, mask_path, img_A_output_dir, img_B_output_dir, mask_output_dir, copy=False,
                          small_translation=False, nr_small_translation:int=1, zoom=False, nr_zoom:int=1,
                          horizontal_flip=False, vertical_flip=False, rotations=False, rotation_angles:list=None, rotation_center:list=None,
                          noise=False):
    
    image_A = cv2.imread(img_A_path, cv2.IMREAD_GRAYSCALE)
    image_B = cv2.imread(img_B_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    filename = os.path.basename(img_A_path)

    if copy:
        img_A_output_path = os.path.join(img_A_output_dir, filename)
        img_B_output_path = os.path.join(img_B_output_dir, filename)
        mask_output_path = os.path.join(mask_output_dir, filename)
        
        cv2.imwrite(img_A_output_path, image_A)
        cv2.imwrite(img_B_output_path, image_B)
        cv2.imwrite(mask_output_path, mask)
    
    if small_translation:
        for i in range(nr_small_translation):
            img_A_output_path = os.path.join(img_A_output_dir, filename[:-4]+f"_trans{i+1}.png")
            img_B_output_path = os.path.join(img_B_output_dir, filename[:-4]+f"_trans{i+1}.png")
            mask_output_path = os.path.join(mask_output_dir, filename[:-4]+f"_trans{i+1}.png")
            
            tx = np.random.randint(-10, 10)  # Random trnsalation between -5 and +5 pixel
            ty = np.random.randint(-10, 10)
            M = np.float32([[1, 0, tx], [0, 1, ty]])
            translated_A = cv2.warpAffine(image_A, M, (image_A.shape[1], image_A.shape[0]), borderValue=BG_img)
            translated_B = cv2.warpAffine(image_B, M, (image_B.shape[1], image_B.shape[0]), borderValue=BG_img)
            translated_mask = cv2.warpAffine(mask, M, (mask.shape[1], mask.shape[0]), borderValue=BG_mask)
            
            cv2.imwrite(img_A_output_path, translated_A)
            cv2.imwrite(img_B_output_path, translated_B)
            cv2.imwrite(mask_output_path, translated_mask)

    if zoom:
        for i in range(nr_zoom):
            img_A_output_path = os.path.join(img_A_output_dir, filename[:-4]+f"_zoom{i+1}.png")
            img_B_output_path = os.path.join(img_B_output_dir, filename[:-4]+f"_zoom{i+1}.png")
            mask_output_path = os.path.join(mask_output_dir, filename[:-4]+f"_zoom{i+1}.png")

            (h, w) = image_A.shape[:2]
            center = (w // 2, h // 2)
            zoom_factor = np.random.uniform(0.9, 1.1)
            M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
            zoomed_A = cv2.warpAffine(image_A, M, (w, h), borderValue=BG_img)
            zoomed_B = cv2.warpAffine(image_B, M, (w, h), borderValue=BG_img)
            zoomed_mask = cv2.warpAffine(mask, M, (w, h), borderValue=BG_mask)

            cv2.imwrite(img_A_output_path, zoomed_A)
            cv2.imwrite(img_B_output_path, zoomed_B)
            cv2.imwrite(mask_output_path, zoomed_mask)
    
    if horizontal_flip:
        img_A_output_path = os.path.join(img_A_output_dir, filename[:-4]+"_flipH.png")
        img_B_output_path = os.path.join(img_B_output_dir, filename[:-4]+"_flipH.png")
        mask_output_path = os.path.join(mask_output_dir, filename[:-4]+"_flipH.png")

        flipped_A = cv2.flip(image_A, 1)
        flipped_B = cv2.flip(image_B, 1)
        flipped_mask = cv2.flip(mask, 1)

        cv2.imwrite(img_A_output_path, flipped_A)
        cv2.imwrite(img_B_output_path, flipped_B)
        cv2.imwrite(mask_output_path, flipped_mask)

    if vertical_flip:
        img_A_output_path = os.path.join(img_A_output_dir, filename[:-4]+"_flipV.png")
        img_B_output_path = os.path.join(img_B_output_dir, filename[:-4]+"_flipV.png")
        mask_output_path = os.path.join(mask_output_dir, filename[:-4]+"_flipV.png")

        flipped_A = cv2.flip(image_A, 0)
        flipped_B = cv2.flip(image_B, 0)
        flipped_mask = cv2.flip(mask, 0)

        cv2.imwrite(img_A_output_path, flipped_A)
        cv2.imwrite(img_B_output_path, flipped_B)
        cv2.imwrite(mask_output_path, flipped_mask)
        
    if rotations:
        if rotation_angles is None:
            print("rotation angles not defined")
        else:
            for angle in rotation_angles:
                img_A_output_path = os.path.join(img_A_output_dir, filename[:-4]+f"_rot{angle}.png")
                img_B_output_path = os.path.join(img_B_output_dir, filename[:-4]+f"_rot{angle}.png")
                mask_output_path = os.path.join(mask_output_dir, filename[:-4]+f"_rot{angle}.png")
                
                (h, w) = image_A.shape[:2]
                if rotation_center is None:
                    center = (w // 2, h // 2)
                else:
                    center=rotation_center
                scale = 1.0
                M = cv2.getRotationMatrix2D(center, angle, scale)
                rotated_A = cv2.warpAffine(
                    image_A, M, (w, h),
                    flags=cv2.INTER_LINEAR,
                    borderMode=cv2.BORDER_CONSTANT,
                    borderValue=(BG_img, BG_img, BG_img) if len(image_A.shape) == 3 else BG_img
                )
                rotated_B = cv2.warpAffine(
                    image_B, M, (w, h),
                    flags=cv2.INTER_LINEAR,
                    borderMode=cv2.BORDER_CONSTANT,
                    borderValue=(BG_img, BG_img, BG_img) if len(image_A.shape) == 3 else BG_img
                )
                rotated_mask = cv2.warpAffine(
                    mask, M, (w, h),
                    flags=cv2.INTER_LINEAR,
                    borderMode=cv2.BORDER_CONSTANT,
                    borderValue=(BG_mask, BG_mask, BG_mask) if len(mask.shape) == 3 else BG_mask
                )

                cv2.imwrite(img_A_output_path, rotated_A)
                cv2.imwrite(img_B_output_path, rotated_B)
                cv2.imwrite(mask_output_path, rotated_mask)

    if noise:
        img_A_output_path = os.path.join(img_A_output_dir, filename[:-4]+"_noise.png")
        img_B_output_path = os.path.join(img_B_output_dir, filename[:-4]+"_noise.png")
        mask_output_path = os.path.join(mask_output_dir, filename[:-4]+"_noise.png")

        noisy_image_A = image_A.copy()
        mean = 0
        stddev = 0.5  # noise level
        added_noise = np.random.normal(mean, stddev, image_A.shape).astype(np.uint8)
        noisy_image_A = cv2.add(image_A, added_noise)
        noisy_image_B = cv2.add(image_B, added_noise)

        cv2.imwrite(img_A_output_path, noisy_image_A)
        cv2.imwrite(img_B_output_path, noisy_image_B)
        cv2.imwrite(mask_output_path, mask)


In [ ]:
input_dir = "Dataset_ROBOFLOW_cropped"
output_dir = "Dataset_ROBOFLOW_augmented"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)
for folder in ["A", "B", "label", "list"]:
    fold_path = os.path.join(output_dir, folder).replace('\\', '/')
    os.makedirs(fold_path, exist_ok=True)


for filename in os.listdir(os.path.join(input_dir,"A")):
    file_paths = {}
    output_sub_dirs = {}
    for folder in ["A", "B", "label"]:
        file_paths[folder] = os.path.join(input_dir, folder, filename).replace('\\', '/')
        output_sub_dirs[folder] = os.path.join(output_dir, folder).replace('\\', '/')

    if "test" in filename[:5]: # in test folder no augmentations
        save_and_apply_transformations(img_A_path=file_paths["A"], img_B_path=file_paths["B"], mask_path=file_paths["label"],
                                img_A_output_dir=output_sub_dirs["A"], img_B_output_dir=output_sub_dirs["B"], mask_output_dir=output_sub_dirs["label"],
                                copy=True)
        continue
    
    # For each of the images in the folder create 3 images with different rotations
    rotation_angles = [90,180,270]
    save_and_apply_transformations(img_A_path=file_paths["A"], img_B_path=file_paths["B"], mask_path=file_paths["label"],
                                   img_A_output_dir=output_sub_dirs["A"], img_B_output_dir=output_sub_dirs["B"], mask_output_dir=output_sub_dirs["label"],
                                   copy=True, rotations=True, rotation_angles=rotation_angles)

In [5]:
output_dir = "Dataset_ROBOFLOW_augmented"

update_filenames_list(input_folder=os.path.join(output_dir, 'A'), output_folder=os.path.join(output_dir, 'list'))